In [ ]:
from dask.distributed import Client, LocalCluster
import bokeh
import geopandas as gpd
import scipy
import libpysal
import momepy
import json
from dask import delayed
import glob
import dask
import dask.dataframe as dd
import momepy
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import json
from tqdm import tqdm
from shapely.geometry import Point
import networkx as nx
import pickle

In [ ]:
daskCluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='100GB')

client = Client(daskCluster)

client

In [ ]:
# with open('./out/singapore/what_cells_are_in_what_cluster_dict.json', 'r') as f:
#     includes_dict = json.load(f)
    
tessellation = gpd.read_parquet("./out/singapore/tessellation.pq")

buildings = gpd.read_parquet("./out/singapore/buildings.pq")

streets = gpd.read_parquet("./out/singapore/streets.pq").explode()

In [ ]:
streets['nID'] = momepy.unique_id(streets)
# buildings['nID'] = momepy.get_network_id(buildings, streets,
#                                                'nID').values

# buildings = buildings.drop_duplicates("uID").drop(columns="index_right")
# tessellation = tessellation.merge(buildings[['uID', 'nID']], on='uID', how='left')

In [ ]:
### Distance Weight
dist200 = libpysal.weights.DistanceBand.from_dataframe(buildings, 200, ids='uID')

In [ ]:
# @dask.delayed
# def queen_contiguity_load(n1, includes_dict):
#     chunk = gpd.read_parquet(f"./out/singapore/expanded_cells_in_chunk_{int(n1)}.pq")
    
#     # weights
    # queen_1 = libpysal.weights.w_subset(momepy.sw_high(k=1, gdf=chunk, ids='uID'), includes_dict[str(n1)]).to_adjlist()
    # queen_3 = libpysal.weights.w_subset(momepy.sw_high(k=3, gdf=chunk, ids='uID'), includes_dict[str(n1)]).to_adjlist()
    
#     return (queen_1, queen_3)

In [ ]:
# file_pattern = "./out/singapore/expanded_cells_in_chunk_*.pq"
# file_list = glob.glob(file_pattern)
# num_files = len(file_list)

# weight_out = [process(n1, includes_dict) for n1 in tqdm(range(num_files))]

# weight_out = [queen_contiguity_load(n1, includes_dict) for n1 in range(num_files)]

# weight_out = dask.compute(weight_out)


In [ ]:
# queen_1 = libpysal.weights.W.from_adjlist(pd.concat(weight_out[0][i][0] for i in range(num_files)))

# queen_3 = libpysal.weights.W.from_adjlist(pd.concat(weight_out[0][i][1] for i in range(num_files)))

In [ ]:
tessellation_bridged = gpd.read_parquet("./out/singapore/tessellation_bridges.pq")

In [10]:
queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tessellation_bridged, ids="uID", silence_warnings=True)
queen_3 = momepy.sw_high(k=3, weights=queen_1)

In [11]:
# # Find the uIDs in tessellation that are not in buildings
# tessellation_uids = set(tessellation['uID'])
# buildings_uids = set(buildings['uID'])
# new_uids = tessellation_uids - buildings_uids

# # Create a new GeoDataFrame with the new uIDs and empty geometry
# new_rows = pd.DataFrame({'uID': list(new_uids)})
# new_rows['geometry'] = None
# new_gdf = gpd.GeoDataFrame(new_rows, crs=buildings.crs)

# # Append the new GeoDataFrame to buildings
# new_buildings = pd.concat([buildings, new_gdf], ignore_index=True)

In [12]:
### Building Area
buildings['area'] = momepy.Area(buildings).series

## Building Shapes

### Circular Compactness
buildings['circular_com'] =  momepy.CircularCompactness(buildings).series

### Elongation
blg_elongation = momepy.Elongation(buildings)
buildings['elongation'] = blg_elongation.series

## Squareness
blg_squareness = momepy.Squareness(buildings)
buildings['squareness'] = blg_squareness.series

buildings['eri'] = momepy.EquivalentRectangularIndex(buildings).series

buildings['area'] = momepy.Area(buildings).series


buildings['orientation'] = momepy.Orientation(buildings).series

  0%|          | 0/76533 [00:00<?, ?it/s]

/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/shape.py:962: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == "Polygon":
/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/shape.py:956: RuntimeWarning: invalid value encountered in arccos
  angle = np.degrees(np.arccos(cosine_angle))
/home/virgilxw/miniconda3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/virgilxw/miniconda3/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


  0%|          | 0/76533 [00:00<?, ?it/s]

/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/distribution.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type in ["Polygon", "MultiPolygon", "LinearRing"]:


In [13]:
tessellation["convexity"] = momepy.Convexity(tessellation).series
tessellation["area"] = momepy.Area(tessellation).series

tessellation["neighbours"] = momepy.Neighbors(tessellation, queen_3,'uID').series
tessellation["neighbours400"] = momepy.Neighbors(tessellation, dist200, 'uID').series
tessellation["covered_area"] = momepy.CoveredArea(tessellation, queen_1, "uID", verbose=False).series

areas = momepy.Area(tessellation).series
tessellation["mean_area"] = momepy.AverageCharacter(
tessellation, values=areas, spatial_weights=queen_3, unique_id='uID').mean

tessellation["mean_area_iq"] = momepy.AverageCharacter(
    tessellation, areas, queen_3, 'uID', rng=(25, 75)).mean

tessellation["area_rng"] = momepy.Range(tessellation, values='area',
                                        spatial_weights=queen_3, unique_id='uID').series
tessellation["area_iqr"] = momepy.Range(tessellation, values='area',
                        spatial_weights=queen_3, unique_id='uID',
                        rng=(25, 75)).series
tessellation["rea_theil"] = momepy.Theil(tessellation, values='area',
                        spatial_weights=queen_3,
                        unique_id='uID').series

tessellation["shannon"] = momepy.Shannon(tessellation, values='area',spatial_weights=queen_3, unique_id='uID').series


  0%|          | 0/76498 [00:00<?, ?it/s]

  0%|          | 0/76498 [00:00<?, ?it/s]

  0%|          | 0/76498 [00:00<?, ?it/s]

  0%|          | 0/76498 [00:00<?, ?it/s]

  0%|          | 0/76498 [00:00<?, ?it/s]

  0%|          | 0/76498 [00:00<?, ?it/s]

  0%|          | 0/76498 [00:00<?, ?it/s]

  0%|          | 0/76498 [00:00<?, ?it/s]

In [14]:
edg_lin = momepy.Linearity(streets)
streets['linearity'] = edg_lin.series

profile = momepy.StreetProfile(streets, buildings)
streets["width"] = profile.w
streets["width_deviation"] = profile.wd
streets["openness"] = profile.o

/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/shape.py:1286: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == "LineString"


In [15]:
# primal = momepy.gdf_to_nx(streets, approach='primal')

In [16]:
%%time
# momepy.node_degree(primal)

CPU times: user 0 ns, sys: 3 µs, total: 3 µs
Wall time: 4.53 µs


In [17]:
%%time
# primal = momepy.closeness_centrality(primal, radius=400, name='closeness400', distance='mm_len', weight='mm_len')

CPU times: user 0 ns, sys: 2 µs, total: 2 µs
Wall time: 4.77 µs


In [18]:
%%time
# primal = momepy.meshedness(primal, radius=400, distance="mm_len")

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.25 µs


In [19]:
# %%time
# primal = momepy.straightness_centrality(primal)

In [20]:
# %%time
# primal = momepy.closeness_centrality(primal, name='closeness_global', weight='mm_len')

In [21]:
# %%time
# primal =  momepy.betweenness_centrality(primal, name='betweenness_metric_n', mode='nodes', weight='mm_len', k=100)

In [22]:
# %%time
# primal = momepy.straightness_centrality(primal)
# # momepy.mean_nodes(primal, 'straightness')
# momepy.mean_nodes(primal, 'closeness400')
# # momepy.mean_nodes(primal, 'closeness_global')
# # momepy.mean_nodes(primal, 'betweenness_metric_n')

# nodes, streets = momepy.nx_to_gdf(primal)

In [23]:
# dual = momepy.gdf_to_nx(streets, approach='dual')

# dual = momepy.closeness_centrality(dual, radius=5, name='angcloseness400', weight='angle')

# # dual = momepy.closeness_centrality(dual, name='closeness_global_ang', weight='angle')

# # dual = momepy.betweenness_centrality(dual, name='angbetweenness', mode='nodes', weight='angle')

# edges_d = momepy.nx_to_gdf(dual)


In [24]:
# Find the uIDs in tessellation that are not in buildings
tessellation_uids = set(tessellation['uID'])
buildings_uids = set(buildings['uID'])
new_uids = tessellation_uids - buildings_uids

# Create a new GeoDataFrame with the new uIDs and empty geometry
new_rows = pd.DataFrame({'uID': list(new_uids)})
new_rows['geometry'] = None
new_gdf = gpd.GeoDataFrame(new_rows, crs=buildings.crs)

# Append the new GeoDataFrame to buildings
buildings = pd.concat([buildings, new_gdf], ignore_index=True)

In [25]:
circular_compactness = momepy.CircularCompactness(buildings)
buildings['weighted_circom'] = momepy.WeightedCharacter(
buildings, circular_compactness.series, queen_3, 'uID', momepy.Area(buildings).series).series

buildings['neighbour_dist'] = momepy.NeighborDistance(buildings, queen_1, 'uID').series

buildings['interbuilding_distance'] = momepy.MeanInterbuildingDistance(buildings, queen_1, 'uID', 3, verbose=False).series

interblg_distance = momepy.MeanInterbuildingDistance(
buildings, queen_1, 'uID', spatial_weights_higher=queen_3)
buildings['mean_ib_dist'] = interblg_distance.series

buildings['mean_ib_dist'] = momepy.MeanInterbuildingDistance(
    buildings, queen_1, 'uID', order=3).series




  0%|          | 0/76533 [00:00<?, ?it/s]

  0%|          | 0/76533 [00:00<?, ?it/s]

Computing mean interbuilding distances...


  0%|          | 0/76533 [00:00<?, ?it/s]

Computing mean interbuilding distances...


  0%|          | 0/76533 [00:00<?, ?it/s]

In [26]:
cols = list(streets.columns)
cols.remove('nID')
cols.remove('geometry')

In [27]:
buildings['weighted_circom'] = momepy.WeightedCharacter(
buildings, circular_compactness.series, queen_3, 'uID', momepy.Area(buildings).series).series

  0%|          | 0/76533 [00:00<?, ?it/s]

In [28]:
streets_dropped = streets.drop(cols,axis=1).reset_index(drop=True)

In [29]:
buildings = buildings.set_crs("epsg:3414")

In [30]:
buildings = gpd.sjoin_nearest(left_df=buildings, right_df=streets_dropped, how='left', max_distance=1000).drop_duplicates(subset=['geometry'])
duplicates = buildings[buildings.duplicated(['uID'], keep=False)]
if len(duplicates) > 0:
    print(f"There are {len(duplicates)} rows with duplicate uids.")
    print(duplicates)
else:
    print("There are no rows with duplicate uids.")


There are no rows with duplicate uids.


In [31]:
buildings = buildings.dropna(subset=['geometry'])
tessellation = tessellation.dropna(subset=['geometry'])

In [32]:
buildings['orientation'] = momepy.Orientation(buildings).series
tessellation['orientation'] = momepy.Orientation(tessellation).series

  0%|          | 0/76533 [00:00<?, ?it/s]

/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/distribution.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type in ["Polygon", "MultiPolygon", "LinearRing"]:


  0%|          | 0/76498 [00:00<?, ?it/s]

In [33]:
str_align = momepy.StreetAlignment(buildings, streets,
                                   'orientation', 'nID',
                                   'nID')
buildings['str_align'] = str_align.series

/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/distribution.py:84: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif geom.type in ["LineString", "MultiLineString"]:


In [34]:
blg_cell_align = momepy.CellAlignment(buildings, tessellation,
                                      'orientation', 'orientation',
                                      'uID', 'uID')
buildings['cell_align'] = blg_cell_align.series

In [35]:
tessellation['car'] = momepy.AreaRatio(tessellation, buildings, 'area', 'area', 'uID').series

In [36]:
nodes = gpd.read_parquet("./out/singapore/vertices_frame.pq")
edges = gpd.read_parquet("./out/singapore/edges_frame.pq")

In [37]:
buildings = buildings.drop(["nID", "index_right"], axis=1)
nodes = nodes.drop(["vertID"], axis=1)
edges = edges.drop("edgeID", axis=1)

In [38]:
buildings = gpd.sjoin_nearest(left_df=buildings, right_df=nodes, how='left', max_distance=1000).drop_duplicates(subset=['geometry'])
duplicates = buildings[buildings.duplicated(['uID'], keep=False)]
if len(duplicates) > 0:
    print(f"There are {len(duplicates)} rows with duplicate uids.")
    print(duplicates)
else:
    print("There are no rows with duplicate uids.")


There are no rows with duplicate uids.


In [39]:
buildings = buildings.drop("index_right", axis=1)
buildings = gpd.sjoin_nearest(left_df=buildings, right_df=edges, how='left', max_distance=1000).drop_duplicates(subset=['geometry'])
duplicates = buildings[buildings.duplicated(['uID'], keep=False)]
if len(duplicates) > 0:
    print(f"There are {len(duplicates)} rows with duplicate uids.")
    print(duplicates)
else:
    print("There are no rows with duplicate uids.")


There are no rows with duplicate uids.


In [40]:
tessellation = tessellation.merge(buildings, how='left', left_on = "uID", right_on="uID" )
tessellation = tessellation.drop(['eID', "geometry_y", "index_right"], axis=1)
tessellation = tessellation.rename(columns={"geometry_x": "geometry"})
tessellation = gpd.GeoDataFrame(tessellation.dropna(subset=['geometry']))

In [41]:
with open('./out/singapore/weights.pickle', 'wb') as f:
    pickle.dump(queen_1, f)
    pickle.dump(queen_3, f)
    pickle.dump(dist200, f)
    f.close()

In [42]:
print(tessellation)

           uID                                           geometry  convexity  \
0      75707.0  POLYGON ((45510.911 33224.180, 45517.076 33236...   0.957765   
1      74906.0  MULTIPOLYGON (((43852.067 36248.967, 43784.590...   0.840004   
2      74909.0  MULTIPOLYGON (((43767.188 36235.006, 43757.647...   0.982273   
3      74915.0  MULTIPOLYGON (((43800.984 36416.304, 43801.539...   0.941257   
4      74921.0  POLYGON ((43453.691 36562.605, 43453.334 36563...   0.921910   
...        ...                                                ...        ...   
76493  74471.0  POLYGON ((42502.870 37070.330, 42522.949 37069...   0.936044   
76494  75817.0  POLYGON ((44572.445 35487.575, 44678.812 35737...   0.653799   
76495  75833.0  MULTIPOLYGON (((45462.523 38178.956, 45451.205...   0.244914   
76496  75840.0  POLYGON ((45232.729 37907.457, 45238.212 37858...   0.991706   
76497  75848.0  POLYGON ((45265.010 37965.610, 45253.833 37953...   1.000000   

              area_x  neighbours  neigh

In [43]:
tessellation.to_parquet("./out/singapore/tessellation_stats.pq")

In [44]:
# chunk_size = 250000
# chunks = []
# for i in range(0, len(tessellation), chunk_size):
#     chunks.append(tessellation.iloc[i:i + chunk_size])

In [45]:
# import os
# new_directory = "./out/singapore/tessellation/"

# if not os.path.exists(new_directory):
#     os.makedirs(new_directory)
#     print(f"Created directory: {new_directory}")
# else:
#     print(f"Directory already exists: {new_directory}")

# # iterate over all files in the directory and delete them
# for filename in os.listdir(new_directory):
#     file_path = os.path.join(new_directory, filename)
#     if os.path.isfile(file_path):
#         os.remove(file_path)
#         print(f"Deleted file: {file_path}")

In [46]:
# for i, chunk in enumerate(chunks):
#     chunk.to_parquet(f"./out/singapore/tessellation/tesselation_chunk_{i}.pq")

In [47]:
# duplicates = tessellation[tessellation.duplicated(['uID'], keep=False)]
# if len(duplicates) > 0:
#     print(f"There are {len(duplicates)} rows with duplicate uids.")
#     print(duplicates)
# else:
#     print("There are no rows with duplicate uids.")

In [48]:
# import pickle

# # Open the file for reading
# with open('./out/singapore/weights.pickle', 'rb') as f:
#     # Load the variables in the same order that they were dumped
#     queen_1 = pickle.load(f)
#     queen_3 = pickle.load(f)
#     dist200 = pickle.load(f)
#     tessellation = pickle.load(f)

In [49]:
# tessellation.to_parquet("./out/singapore/tessellation_with_data.pq", write_metadata_file=False)